In [19]:
import numpy as np
import torch

In [20]:
torch.__version__

'1.8.0+cu101'

In [5]:
torch.cuda.is_available()

False

1. Tensors (torch.Tensor, Tensor Attributes)

In [6]:
#from python list or sequence
a = [[1_000, 0], [3, 4]]
#dtype int8, unsigned int8, int16, int32, int64, float16, float32, float64, boolean
x = torch.tensor(a, dtype=torch.bool, device='cpu')
x.dtype, x.device, x.stride() #reference to tensor.Storage

(torch.bool, device(type='cpu'), (2, 1))

In [18]:
# 9 cpu constructors with specific dtype [torch.FloatTensor ...]
# 9 gpu constructors with specific dtype [torch.cuda.FloatTensor ...]

torch.float32

In [ ]:
# from array_like data
a = np.array([[1_000, 0], [3, 4]])
q = torch.as_tensor(a) #avoid copy
q[0, 0] = -1
a[0, 0], q.device
#on gpu this trick isn't work

(-1, device(type='cpu'))

In [ ]:
#from numpy (see bridge section)

In [16]:
#from another tensor
data = torch.tensor([[1, 2], [3, 4]])
x_ones = torch.ones_like(data)

x_rand = torch.rand_like(data, dtype=torch.float16)

x_ones, x_ones.dtype, x_rand, x_rand.dtype

(tensor([[1, 1],
         [1, 1]]), torch.int64, tensor([[0.7793, 0.8838],
         [0.1787, 0.9209]], dtype=torch.float16), torch.float16)

In [8]:
#numpy axis is equal to torch dim
a = np.array([[[1, 2, 3], 
               [3, 4, 0]],
                        [[1, 2, 3], 
                        [3, 4, 0]]])
a.shape, a.sum(axis=0), a.sum(axis=1), a.sum(axis=2)

((2, 2, 3), array([[2, 4, 6],
        [6, 8, 0]]), array([[4, 6, 3],
        [4, 6, 3]]), array([[6, 7],
        [6, 7]]))

In [40]:
b = torch.tensor(a)
b.sum(dim=0), b.sum(dim=1), b.sum(dim=2)

(tensor([[2, 4, 6],
         [6, 8, 0]]), tensor([[4, 6, 3],
         [4, 6, 3]]), tensor([[6, 7],
         [6, 7]]))

In [26]:
# tensor operations
# that have a _ suffix are in-place
#x.copy_(y)
#x.t_()
tensor = torch.ones((3, 3))
tensor.add_(5)
tensor

tensor([[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]])

In [27]:
#numpy pytorch bridge
#Tensors on the CPU and NumPy arrays can share their underlying memory locations, and changing one will change the other.
  # torch -> numpy
t = torch.ones(5)
n = t.numpy()
t, n

(tensor([1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1.], dtype=float32))

In [28]:
t.add_(5)
t, n

(tensor([6., 6., 6., 6., 6.]), array([6., 6., 6., 6., 6.], dtype=float32))

In [36]:
  #numpy -> torch
n = np.ones(3)
t = torch.from_numpy(n)
n, t

(array([1., 1., 1.]), tensor([1., 1., 1.], dtype=torch.float64))

In [37]:
np.add(n, 1, out=n)
n, t

(array([2., 2., 2.]), tensor([2., 2., 2.], dtype=torch.float64))

2. torch.Autograd

In [69]:
#torch autograd
  #out = t^2
t = torch.tensor([1, 2, 3], dtype=torch.float32, requires_grad=True)
out = t.pow(2)
out, t.shape

(tensor([1., 4., 9.], grad_fn=<PowBackward0>), torch.Size([3]))

In [70]:
  #When we call .backward() on OUT, autograd calculates these gradients and stores them in the respective tensor's .grad attribute.
  #We need to explicitly pass a gradient argument in OUT.backward() because it is a vector. gradient is a tensor of the same shape as OUT, and it represents the gradient of the OUT itself
out.backward(gradient=torch.tensor([1, 1, 1]))

In [66]:
t.grad

tensor([2., 4., 6.])

In [92]:
  #Equivalently, we can aggregate Q into a scalar and call backward implicitly [grad can be implicitly created only for scalar outputs]
t = torch.tensor([1, 2, 3], dtype=torch.float32, requires_grad=True)
out2 = t.pow(2).sum()
out2

tensor(14., grad_fn=<SumBackward0>)

In [93]:
out2.backward()
t.grad

tensor([2., 4., 6.])

In [94]:
out2.grad_fn

In [ ]:
  # Pytorch support differentiation of a scalar functions (self derivative is 1), os if we want too start backward from a non-scalar value, we provide torch.tensor in backward(gradient=...)

In [99]:
  #Example 2
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2

In [100]:
Q.backward(gradient=torch.tensor([1., 1.]))

In [102]:
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


In [ ]:
  #Generally speaking, torch.autograd is an engine for computing vector-Jacobian product. That is, given any vector v, compute the product J.T * v (see perfect example in doc)

3. Computational Graph

Conceptually, autograd keeps a record of data (tensors) and all executed operations (along with the resulting tensors) in a **directed acyclic graph DAG** consisting of a autograd.Function objects. In this DAG, leaves are the input tensors, roots are the output tensors. By tracking this graph from roots to leaves, you can automatically compute the gradients using the chain rule.


In a forward pass, autograd does twoo things simultaneously:
  - run the requested operation to compute a resulting tensor
  - maintain the operation's gradient function in the DAG

The backward pass kicks off when .backward is called on the DAG root. autograd then:
  - computes the gradients from each .grad_fn
  - accumulates them in the respective tensor's .grad attribute
  - using the chain rule, propagates all the way to the leaf tensors

**DAGs are dynamic in PyTorch and, after .backward() call, autograd starts populate a new graph.**

In [ ]:
  #Exclusion from the DAG - requires_grad = False (frozen parameters - we dont need their parameters)
  #is important for funetuning a pretrained network

In [104]:
from torch import nn, optim
import torchvision

model = torchvision.models.resnet18(pretrained=True)

#freeze all the parameters in the network
for param in model.parameters():
  param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


let's say we want to finetune the model on a new dataset with 10 label. In resnet, the classifier is the last linear layer model.fc. We can simply replace it with a new linear layer (unfrose by default)

In [106]:
model.fc = nn.Linear(512, 10)

In [107]:
#optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)
#same functionality is available as a context manager in torch.no_grad()

4. Neural network